<a href="https://colab.research.google.com/github/annvorosh/GB/blob/ETL/ETL_L06_airflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Урок 6. Операторы в Airflow и их применение для ETL

1. Создайте новый граф. Добавьте в него BashOperator, который будет генерировать рандомное число и печатать его в
консоль.

2. Создайте PythonOperator, который генерирует рандомное число, возводит его в квадрат и выводит в консоль исходное число и результат.

3. Сделайте оператор, который отправляет запрос к https://goweather.herokuapp.com/weather/"location" (вместо location используйте ваше местоположение).

4. Задайте последовательный порядок выполнения операторов.

In [2]:
!pip install apache-airflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.3/511.3 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 7.1 MB/

In [15]:
!pip install requests

In [14]:
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta
import random
import requests

# Определите аргументы DAG
default_args = {
    'owner': 'airflow',
    'start_date': datetime(2024, 1, 1),
    'depends_on_past': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Инициализируйте DAG
dag = DAG(
    'example_etl_dag',
    default_args=default_args,
    schedule=timedelta(days=1),  # Задайте интервал выполнения DAG
)

# Оператор, который генерирует рандомное число и выводит его в консоль
generate_random_number = BashOperator(
    task_id='generate_random_number',
    bash_command='echo $(( $RANDOM % 100 ))',
    dag=dag,
)

# Оператор, который генерирует рандомное число, возводит его в квадрат и выводит в консоль
def square_and_print_number():
    random_number = random.randint(1, 100)
    squared_number = random_number ** 2
    print(f"Original number: {random_number}, Squared number: {squared_number}")

generate_and_square_number = PythonOperator(
    task_id='generate_and_square_number',
    python_callable=square_and_print_number,
    dag=dag,
)

# Оператор, отправляющий запрос к API погоды
def get_weather(location, **kwargs):
    location = kwargs.get('location', 'Vyborg')
    response = requests.get(f'https://goweather.herokuapp.com/weather/{location}')
    result = response.json()
    print(f"Weather in {location}: {result}")
    return result


get_weather_operator = PythonOperator(
    task_id='get_weather',
    python_callable=get_weather,
    op_args=['location'],
    dag=dag,
)

# порядок выполнения операторов
generate_random_number >> generate_and_square_number >> get_weather_operator


<Task(PythonOperator): get_weather>

In [10]:
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
import random
import requests

# Установка аргументов для DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2024, 1, 1),
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Создание DAG
dag = DAG(
    'my_dag',
    default_args=default_args,
    description='A simple DAG',
    schedule_interval=timedelta(days=1),
)

# Оператор Bash
t1 = BashOperator(
    task_id='5',
    bash_command='echo {{ task_instance.xcom_pull(task_ids="generate_random_number") }}',
    dag=dag,
)

# Оператор Python
def generate_random_number(**kwargs):
    result = random.randint(1, 100)
    print(f"Generated random number: {result}")
    return result

t2 = PythonOperator(
    task_id='generate_random_number',
    python_callable=generate_random_number,
    provide_context=True,
    dag=dag,
)

# Оператор Python для отправки запроса
def get_weather(**kwargs):
    location = kwargs.get('location', 'your_location')
    response = requests.get(f'https://goweather.herokuapp.com/weather/{location}')
    result = response.text
    print(f"Weather in {location}: {result}")
    return result

t3 = PythonOperator(
    task_id='get_weather',
    python_callable=get_weather,
    provide_context=True,
    dag=dag,
)

# Устанавливаем порядок выполнения
t2 >> t1
t3 >> t1


<ipython-input-10-81e8f441d01c>:2 DeprecationWarning: The `airflow.operators.bash_operator.BashOperator` class is deprecated. Please use `'airflow.operators.bash.BashOperator'`.

<ipython-input-10-81e8f441d01c>:3 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

<ipython-input-10-81e8f441d01c>:20 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

<ipython-input-10-81e8f441d01c>:40 RemovedInAirflow3Warning: provide_context is deprecated as of 2.0 and is no longer required

<ipython-input-10-81e8f441d01c>:55 RemovedInAirflow3Warning: provide_context is deprecated as of 2.0 and is no longer required

<Task(BashOperator): 5>